[[이유한님] 캐글 코리아 캐글 스터디 커널 커리큘럼](https://kaggle-kr.tistory.com/32)

[Statoil/C-CORE Iceberg Classifier Challenge](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge)

[Submarineering.EVEN BETTER PUBLIC SCORE until now.](https://www.kaggle.com/code/submarineering/submarineering-even-better-public-score-until-now)

This is **EVEN better the public score** kernel in the competition until now. I wanted strongly dedicate this kernel to that **scavenger colleagues** that spend the time tracking back the work of other members just to take chance and get any good idea only for his own benefit, but, however they don't share anything, just providing bad comments for those who share with the best will and without breaking the rules.

If you do not feel alluded, **please VOTE me UP**.

__DeepL 번역__  

이것은 지금까지의 경쟁에서 **공개 점수** 커널보다 훨씬 더 나은 커널입니다. 저는 이 커널을 다른 회원의 작업을 추적하여 기회를 잡고 자신의 이익을 위해서만 좋은 아이디어를 얻으려고 시간을 소비하지만, 아무것도 공유하지 않고 선한 의지로 규칙을 어기지 않고 공유하는 사람들에게 나쁜 댓글 만 제공하는 **청소부 동료**에게 강력하게 헌정하고 싶었습니다.

암시라고 느끼지 않으시면 **투표해 주세요**.

In [1]:
import os
import numpy as np
import pandas as pd
from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))  # 윈도우 사용으로 주석 처리

First thing first@

# Credits to the following awesome authors and kernels

Author: QuantScientist  
File: sub_200_ens_densenet.csv  
Link: https://www.kaggle.com/solomonk/pytorch-cnn-densenet-ensemble-lb-0-1538

Author: wvadim  
File: sub_TF_keras.csv  
Link: https://www.kaggle.com/wvadim/keras-tf-lb-0-18

Author: Ed Miller  
File: sub_fcn.csv  
Link: https://www.kaggle.com/bluevalhalla/fully-convolutional-network-lb-0-193

Author: Chia-Ta Tsai  
File: sub_blend009.csv  
Link: https://www.kaggle.com/cttsai/ensembling-gbms-lb-203

Author: DeveshMaheshwari  
File: sub_keras_beginner.csv  
Link: https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d

Author: Submarineering  
Files: submission38.csv , submission43.csv  
Link : https://www.kaggle.com/submarineering/submission38-lb01448

Without their truly dedicated efforts, this notebook will not be possible.

# Data Load


In [2]:
sub_path = "../input/statoil-iceverg-submissions"
sub_path = "./input/005_statoil-iceberg-classifier-challenge"   # 내 경로
all_files = os.listdir(sub_path)
all_files.append('submission38.csv')
all_files.append('submission43.csv')
all_files

['sample_submission.csv',
 'submission38.csv',
 'submission43.csv',
 'sub_blend009.csv',
 'sub_fcn.csv',
 'sub_keras_beginner.csv',
 'sub_TF_keras.csv',
 'test.json',
 'train.json',
 'submission38.csv',
 'submission43.csv']

In [3]:
# Read and concatenate submissions
# out1 = pd.read_csv("./input/005_statoil-iceberg-classifier-challenge/sub_200_ens_densenet.csv", index_col=0)    # 파일 받는 법을 몰라서 sub_fcn으로 대체
out1 = pd.read_csv("./input/005_statoil-iceberg-classifier-challenge/sub_fcn.csv", index_col=0)
out2 = pd.read_csv("./input/005_statoil-iceberg-classifier-challenge/sub_TF_keras.csv", index_col=0)
out3 = pd.read_csv("./input/005_statoil-iceberg-classifier-challenge/submission38.csv", index_col=0)
out4 = pd.read_csv("./input/005_statoil-iceberg-classifier-challenge/submission43.csv", index_col=0)
concat_sub = pd.concat([out1, out2, out3, out4], axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
0,5941774d,0.008171,0.005586,0.008875,0.005586
1,4023181e,0.638348,0.145927,0.146779,0.031688
2,b20200e4,0.008061,0.000015,0.048646,0.001000
3,e7f018bb,0.999506,0.999914,0.999184,0.999000
4,4371c8c3,0.774739,0.033843,0.277251,0.022151


In [4]:
# check correlation
concat_sub.corr(numeric_only=True)

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
is_iceberg_0,1.000000,0.789509,0.860398,0.821154
is_iceberg_1,0.789509,1.000000,0.923848,0.900871
is_iceberg_2,0.860398,0.923848,1.000000,0.988002
is_iceberg_3,0.821154,0.900871,0.988002,1.000000


In [5]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

In [6]:
# set up cutoff threshold for lower and upper bounds, easy to twist
cutoff_lo = 0.8
cutoff_hi = 0.2

# Mean Stacking

In [7]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv('stack_mean.csv', index=False, float_format='%.6f')

# Median Stacking

In [8]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_median.csv', index=False, float_format='%.6f')

# PushOut + Median Stacking

In [9]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1,
                                    np.where(np.all(concat_sub.iloc[:, 1:6] < cutoff_hi, axis=1),
                                             0, concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_pushout_median.csv',
                                        index=False, float_format='%.6f')

# MinMax + Mean Stacking

In [10]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1),
                                    concat_sub['is_iceberg_max'],
                                    np.where(np.all(concat_sub.iloc[:, 1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'],
                                             concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_mean.csv',
                                        index=False, float_format='%.6f')

# MinMax + Median Stacking

In [11]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1),
                                    concat_sub['is_iceberg_max'],
                                    np.where(np.all(concat_sub.iloc[:, 1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'],
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median.csv',
                                        index=False, float_format='%.6f')

https://www.kaggle.com/code/submarineering/submarineering-even-better-public-score-until-now